In [ ]:
!curl ipinfo.io

{
  "ip": "34.70.252.13",
  "hostname": "13.252.70.34.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS15169 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# get latest version of tfds
![ ! -f "pip_installed" ] && pip install tensorflow-datasets==4.4.0
!touch pip_installed

     |████████████████████████████████| 4.0 MB 2.3 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 2.3 MB/s 


In [ ]:
# clone repository to contents/TeamSemiSuperCV
!git clone https://github.com/TeamSemiSuperCV/semi-super.git /contents/TeamSemiSuperCV

Cloning into '/contents/TeamSemiSuperCV'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 274 (delta 148), reused 209 (delta 91), pack-reused 0
Receiving objects: 100% (274/274), 87.73 MiB | 46.35 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [ ]:
# change dir to TeamSemiSuperCV (repo code)
import os
os.chdir('/contents/TeamSemiSuperCV')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
!git branch

* main


## Set the params and run the command below.
---



In [ ]:
model_folder = 'reborn_again_500_1e4'

model_dir = 'gs://simclrv2-sb/'

class simclrCommand():
  def __init__(self, model_dir, model_folder, params={}):
    #default params
    self.params = {'train_mode': "pretrain", 
          "mode": "train_then_eval",   
          "resnet_depth": 50,
          "width_multiplier": 2,
          "sk_ratio":0.0625,  
          "color_jitter_strength": 0.5,
          'model_dir': model_dir+model_folder,
          "data_dir": model_dir,
          "use_tpu": True}
    # add in or change params
    if len(params) > 0:
      for k,v in params.items():
        self.params[k]=v
  def compile_command(self):
    simclr_command = ['python run.py']
    for k,v in self.params.items():
      simclr_command.append(f'--{k}={v}')
    return (" ").join(simclr_command)
  def run_command(self):
    !{self.compile_command()}

# train the model
params = {"train_epochs": 500,
          'checkpoint_epochs': 10,
          "train_batch_size": 128, 
          "image_size": 224,   
          'dataset': "xray_reborn",
          "learning_rate": 0.20, 
          'weight_decay': 1e-4, 
          "temperature": 0.1,
          "use_blur": True, 
          "area_range_min": 0.50,
          "eval_split": "validation",    
          "eval_per_loop": False} 
slimsk2 = simclrCommand(model_dir, model_folder, params)
slimsk2.run_command()

# test results
slimsk2.params['mode'] = 'eval'
slimsk2.params['eval_split'] = 'test'

slimsk2.run_command()

2021-11-13 05:33:39.968671: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I1113 05:33:41.425931 139664450193280 dataset_info.py:358] Load dataset info from gs://simclrv2-sb/xray_reborn/1.0.0
I1113 05:33:41.711695 139664450193280 dataset_builder.py:351] Reusing dataset xray_reborn (gs://simclrv2-sb/xray_reborn/1.0.0)
I1113 05:33:41.711954 139664450193280 run.py:592] # train examples: 4708
I1113 05:33:41.712184 139664450193280 run.py:593] # train_steps: 18391
I1113 05:33:41.712236 139664450193280 run.py:594] # eval examples: 524
I1113 05:33:41.712292 139664450193280 run.py:595] # eval steps: 3
I1113 05:33:41.736763 139664450193280 remote.py:237] Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
I1113 05:33:41.736995 139664450193280 tpu_strategy_util.py:61] Deallocate tpu buffers before initializing tpu sy